In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree

# Dados NSL-KDD

In [2]:
data = pd.read_csv(r"C:\Users\luang\Downloads\BRACIS\UNSW_2018_IoT_Botnet_Full5p.csv")
x = data.drop (['category', 'subcategory', 'attack'], axis = 1)
y = data ['attack']

In [3]:
data = x.to_numpy()
data_class = y.to_numpy()

# Abordagem Autoencoder-KNN

In [4]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_anomaly = np.array(list(set(np.arange(0, anTrain)) - set(numbers_knn_anomaly)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = anomalyTrain[numbers_autoencoder_anomaly]
    autoencoderTrain_class = np.ones(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(int(var/2), activation=activation)(input_vector)
    en2 = layers.Dense(int(var/4), activation=activation)(en1)
    de1 = layers.Dense(int(var/2), activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)
    
    encoder = keras.Model(input_vector, en2)
    encoded_input = keras.Input(shape=(int(var/4),))
    decoder_layer = autoencoder.layers[-2]
    decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    encoder_layer = encoder.predict(knnTrain)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(encoder_layer, knnTrain_class)
    
    #Teste do Modelo
    encoder_layer = encoder.predict(modelTest)
    predict_knn = neigh.predict(encoder_layer)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(modelTest_class, predict_knn)
    recall_knn[count] = recall_score(modelTest_class, predict_knn)
    precision_knn[count] = precision_score(modelTest_class, predict_knn)
    f1_knn[count] = f1_score(modelTest_class, predict_knn)
    matrix_knn[count] = confusion_matrix(modelTest_class, predict_knn)
    
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    encoder_layer = encoder.predict(knnTrain)
    dt = tree.DecisionTreeClassifier()
    dt.fit(encoder_layer, knnTrain_class)

    #Teste
    encoder_layer = encoder.predict(modelTest)
    predict_dt = dt.predict(encoder_layer)

    #Métricas
    accuracy_dt[count] = accuracy_score(modelTest_class, predict_dt)
    recall_dt[count] = recall_score(modelTest_class, predict_dt)
    precision_dt[count] = precision_score(modelTest_class, predict_dt)
    f1_dt[count] = f1_score(modelTest_class, predict_dt)
    matrix_dt[count] = confusion_matrix(modelTest_class, predict_dt)
    
    #####################################################
    ####SVM
    #Treinamento SVM
    encoder_layer = encoder.predict(knnTrain)
    clf = svm.SVC(kernel='rbf')
    clf.fit(encoder_layer, knnTrain_class)
    
    #Teste
    encoder_layer = encoder.predict(modelTest)
    predict_svm = clf.predict(encoder_layer)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(modelTest_class, predict_svm)
    recall_svm[count] = recall_score(modelTest_class, predict_svm)
    precision_svm[count] = precision_score(modelTest_class, predict_svm)
    f1_svm[count] = f1_score(modelTest_class, predict_svm)
    matrix_svm[count] = confusion_matrix(modelTest_class, predict_svm)
    
    count = count + 1
    fold_no = fold_no + 1

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 23s 1ms/step - loss: 0.0157
Epoch 2/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0081
Epoch 3/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0078
Epoch 4/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0077
Epoch 5/50
12894/12894 [==========

12894/12894 [==============================] - 20s 2ms/step - loss: 8.5978e-04
Epoch 13/50
12894/12894 [==============================] - 19s 1ms/step - loss: 8.5359e-04
Epoch 14/50
12894/12894 [==============================] - 21s 2ms/step - loss: 8.5015e-04
Epoch 15/50
12894/12894 [==============================] - 22s 2ms/step - loss: 8.4610e-04
Epoch 16/50
12894/12894 [==============================] - 21s 2ms/step - loss: 8.4413e-04
Epoch 17/50
12894/12894 [==============================] - 22s 2ms/step - loss: 8.4241e-04
Epoch 18/50
12894/12894 [==============================] - 24s 2ms/step - loss: 8.4134e-04
Epoch 19/50
12894/12894 [==============================] - 22s 2ms/step - loss: 8.4027e-04
Epoch 20/50
12894/12894 [==============================] - 22s 2ms/step - loss: 8.3583e-04
Epoch 21/50
12894/12894 [==============================] - 21s 2ms/step - loss: 8.3409e-04
Epoch 22/50
12894/12894 [==============================] - 23s 2ms/step - loss: 8.3253e-04
Epoch 23/50

12894/12894 [==============================] - 22s 2ms/step - loss: 0.0126
Epoch 34/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0126
Epoch 35/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0126
Epoch 36/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0126
Epoch 37/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0125
Epoch 38/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0125
Epoch 39/50
12894/12894 [==============================] - 23s 2ms/step - loss: 0.0125
Epoch 40/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0125
Epoch 41/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0125
Epoch 42/50
12894/12894 [==============================] - 23s 2ms/step - loss: 0.0125
Epoch 43/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0125
Epoch 44/50
12894/12894 [==============================

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 26s 2ms/s

Epoch 8/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0292
Epoch 9/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0292
Epoch 10/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0292
Epoch 11/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0292
Epoch 12/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0292
Epoch 13/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0292
Epoch 14/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0291
Epoch 15/50
12894/12894 [==============================] - 25s 2ms/step - loss: 0.0291
Epoch 16/50
12894/12894 [==============================] - 29s 2ms/step - loss: 0.0291
Epoch 17/50
12894/12894 [==============================] - 29s 2ms/step - loss: 0.0291
Epoch 18/50
12894/12894 [==============================] - 25s 2ms/step - loss: 0.0291
Epoch 19/50
12894/12894 [====================

12894/12894 [==============================] - 21s 2ms/step - loss: 0.0080
Epoch 31/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0080
Epoch 32/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0080
Epoch 33/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0080
Epoch 34/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0080
Epoch 35/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0080
Epoch 36/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0080
Epoch 37/50
12894/12894 [==============================] - 22s 2ms/step - loss: 0.0080
Epoch 38/50
12894/12894 [==============================] - 25s 2ms/step - loss: 0.0080
Epoch 39/50
12894/12894 [==============================] - 28s 2ms/step - loss: 0.0080
Epoch 40/50
12894/12894 [==============================] - 24s 2ms/step - loss: 0.0080
Epoch 41/50
12894/12894 [==============================

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 24s 2ms/s

12894/12894 [==============================] - 19s 2ms/step - loss: 0.0027
Epoch 9/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0027
Epoch 10/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0027
Epoch 11/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0027
Epoch 12/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0027
Epoch 13/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 14/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 15/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 16/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 17/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 18/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0026
Epoch 19/50
12894/12894 [==============================]

In [5]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[4.740000e+01 3.000000e-01]
 [4.372800e+03 3.624317e+05]] 
Acurácia:  0.9880793983495826  +/-  0.004257034329311446 
Recall:  0.9880786646524591  +/-  0.004257962650062092 
Precision:  0.9999991738588527  +/-  1.2619570674420804e-06 
F1:  0.993998549265568  +/-  0.0021623210499274034

Autoencoder - SVM:
 [[4.770000e+01 0.000000e+00]
 [6.865700e+03 3.599388e+05]] 
Acurácia:  0.9812848353671655  +/-  0.009821793331015181 
Recall:  0.9812823986758348  +/-  0.009823081445114694 
Precision:  1.0  +/-  0.0 
F1:  0.9905278805004418  +/-  0.005023156909266599

Autoencoder - Decision Tree:
 [[4.680000e+01 9.000000e-01]
 [5.792100e+03 3.610124e+05]] 
Acurácia:  0.9842088997281161  +/-  0.009773916838104389 
Recall:  0.984209300227006  +/-  0.009774323651808357 
Precision:  0.9999974978903033  +/-  2.6197795894529774e-06 
F1:  0.9920159753126081  +/-  0.004997483625159737


In [6]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_IL_IoT.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [7]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_IL_Matrix_IoT.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
